# 프로젝트 : 얼굴을 인식하여 캐릭터 씌우기

# Face Detection VS Face Reconition

### 패키지 설치
 > pip install mediapipe

### 얼굴인식

In [26]:
#라이브러리 설정
import cv2
import mediapipe as mp

#얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection  # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기  or 웹캠 열기 가능 장치 ID
cap = cv2.VideoCapture('face_video.mp4')

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    #model_selection 옵션 0 or 1  0일경우 2m 이내 근거리 모델 인식 1일경우 원거리
    #min_detection_confidence 0.5 이상일 경우 얼굴로 인식한다 임계값 개념 
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)# mediapipe 는 RGB기준으로 처리함
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            #6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀, (귀구슬점, 이주)
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
                print(detection)
                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx = 1, fy =1))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

## 얼굴 인식후 눈, 코, 입에 동그라미 그리기

In [8]:
#라이브러리 설정
import cv2
import mediapipe as mp

#얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection  # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기  or 웹캠 열기 가능 장치 ID
cap = cv2.VideoCapture('face_video.mp4')

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    #model_selection 옵션 0 or 1  0일경우 2m 이내 근거리 모델 인식 1일경우 원거리
    #min_detection_confidence 0.5 이상일 경우 얼굴로 인식한다 임계값 개념 
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)# mediapipe 는 RGB기준으로 처리함
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            #6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀, (귀구슬점, 이주)
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
                #print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] #오른쪽 눈
                left_eye = keypoints[1] #왼쪽 눈
                nose_tip = keypoints[2] #코 끝부분
                
                h, w, _ = image.shape # height, width, channel : 이미지로부터 세로, 가로 크기 가져옴
                right_eye = (int(right_eye.x * w), int(right_eye.y * h)) #이미지 내에서 실제 좌표(x,y)
                left_eye = (int(left_eye.x * w), int(left_eye.y * h))
                nose_tip = (int(nose_tip.x *w), int(nose_tip.y * h))
                
                
                #양 눈에 동그라미 그리기
                cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA)
                cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA)
                # 코에 동그라미 그리기
                cv2.circle(image, nose_tip, 50, (0, 255, 255), 10, cv2.LINE_AA)
                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx = 1, fy =1))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

## 얼굴 인식 후 이미지 입히기

In [2]:
#라이브러리 설정
import cv2
import mediapipe as mp

# 해결 코드 : ValueError: could not broadcast input array from shape (100,100,4) into shape (100,100,3)
def overlay(image, x, y, w, h, overlay_image) : #대상 이미지, x, y 좌표, width , height, 덮어 씌울 이미지(4채널)
    alpha = overlay_image[:,:,3] #BGRA
    mask_image = alpha/255 #0~255 를 255로 나누면 0~1사이의 값 (1: 불투명, 0: 완전)
    #색이 있는 영역은 대상이미지를 사용하고 색이 없는 영역(흰색)은 overlay이미지 사용
    for c in range(0,3) : # channel BGR
        image[y-h : y+h, x-w: x+w, c] = (overlay_image[:,:,c]*mask_image)+(image[y-h:y+h, x-w:x+w, c]*(1-mask_image))
        

#얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection  # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

#이미지 불러오기
image_right_eye = cv2.imread('right_eye.png',cv2.IMREAD_UNCHANGED) #100 x 100
image_left_eye = cv2.imread('left_eye.png',cv2.IMREAD_UNCHANGED) #100 x 100
image_nose = cv2.imread('nose.png',cv2.IMREAD_UNCHANGED) #300 x 100


with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    #model_selection 옵션 0 or 1  0일경우 2m 이내 근거리 모델 인식 1일경우 원거리
    #min_detection_confidence 0.5 이상일 경우 얼굴로 인식한다 임계값 개념 
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)# mediapipe 는 RGB기준으로 처리함
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            #6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀, (귀구슬점, 이주)
            for detection in results.detections:
                #mp_drawing.draw_detection(image, detection) #사각형 특징점 그리기
                #print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] #오른쪽 눈
                left_eye = keypoints[1] #왼쪽 눈
                nose_tip = keypoints[2] #코 끝부분
                
                
                h, w, _ = image.shape # height, width, channel : 이미지로부터 세로, 가로 크기 가져옴
                right_eye = (int(right_eye.x * w)-30, int(right_eye.y * h)-100) #이미지 내에서 실제 좌표(x,y)
                left_eye = (int(left_eye.x * w)-30, int(left_eye.y * h)-100)
                nose_tip = (int(nose_tip.x *w), int(nose_tip.y * h))
                
                overlay(image, *right_eye,50, 50,image_right_eye)
                overlay(image, *left_eye, 50, 50,image_left_eye)
                overlay(image, *nose_tip, 150, 50,image_nose)
                
                
                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx = 0.5, fy =0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()